In [2]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64 # Batch size for training.
epochs = 100 # Number of epochs to train for.
latent_dim = 256 # Latent dimensionality of the encoding space.
num_samples = 10000 # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'kor-eng/kor.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
  input_text, target_text = line.split('\t')
  # We use "tab" as the "start sequence" character
  # for the targets, and "\n" as "end sequence" character.
  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)
      
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
  [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
  [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
  (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
  dtype='float32')
decoder_input_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
  dtype='float32')
decoder_target_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
  dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.
  for t, char in enumerate(target_text):
    # decoder_target_data is ahead of decoder_input_data by one timestep
    decoder_input_data[i, t, target_token_index[char]] = 1.
    if t > 0:
      # decoder_target_data will be ahead by one timestep
      # and will not include the start character.
      decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state= True, return_sequences=True)
encoder2 = LSTM(latent_dim, return_state=True, return_sequences=True)
encoder3 = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder3(encoder2(encoder(encoder_inputs)))

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder2 = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder3 = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder3(decoder2(decoder(decoder_inputs, initial_state=encoder_states)))
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
     batch_size=batch_size,
     epochs=epochs,
     validation_split=0.2)
# Save model
model.save('s2s3.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder(decoder_inputs, initial_state=decoder_states_inputs)
decoder2_outputs, state_h2, state_c2 = decoder2(decoder(decoder_inputs, initial_state=[state_h, state_c]))
decoder3_outputs, state_h3, state_c3 = decoder3(decoder2(decoder(decoder_inputs, initial_state=[state_h, state_c])))
decoder_states = [state_h3, state_c3]
decoder_outputs = decoder_dense(decoder3_outputs)
decoder_model = Model(
  [decoder_inputs] + decoder_states_inputs,
  [decoder_outputs] + decoder_states)
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
  (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
  (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
  # Encode the input as state vectors.
  states_value = encoder_model.predict(input_seq)

  # Generate empty target sequence of length 1.
  target_seq = np.zeros((1, 1, num_decoder_tokens))
  # Populate the first character of target sequence with the start character.
  target_seq[0, 0, target_token_index['\t']] = 1.

  # Sampling loop for a batch of sequences
  # (to simplify, here we assume a batch of size 1).
  stop_condition = False
  decoded_sentence = ''
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict(
      [target_seq] + states_value)

    # Sample a token
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = reverse_target_char_index[sampled_token_index]
    decoded_sentence += sampled_char

    # Exit condition: either hit max length
    # or find stop character.
    if (sampled_char == '\n' or
      len(decoded_sentence) > max_decoder_seq_length):
      stop_condition = True

    # Update the target sequence (of length 1).
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.

    # Update states
    states_value = [h, c]

  return decoded_sentence


for seq_index in range(100):
  # Take one sequence (part of the training set)
  # for trying out decoding.
  input_seq = encoder_input_data[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  print('-')
  print('Input sentence:', input_texts[seq_index])
  print('Decoded sentence:', decoded_sentence)

Number of samples: 799
Number of unique input tokens: 69
Number of unique output tokens: 632
Max sequence length for inputs: 124
Max sequence length for outputs: 54
Train on 639 samples, validate on 160 samples
Epoch 1/100
639/639 [==============================] - 51s 80ms/step - loss: 1.3103 - val_loss: 2.0751
Epoch 2/100
639/639 [==============================] - 44s 69ms/step - loss: 1.1104 - val_loss: 2.0247
Epoch 3/100
639/639 [==============================] - 46s 72ms/step - loss: 1.0918 - val_loss: 2.0187
Epoch 4/100
639/639 [==============================] - 46s 72ms/step - loss: 1.0814 - val_loss: 2.0737
Epoch 5/100
639/639 [==============================] - 47s 74ms/step - loss: 1.0752 - val_loss: 2.0572
Epoch 6/100
639/639 [==============================] - 47s 74ms/step - loss: 1.0685 - val_loss: 2.0996
Epoch 7/100
639/639 [==============================] - 48s 75ms/step - loss: 1.0646 - val_loss: 2.1365
Epoch 8/100
639/639 [==============================] - 51s 80ms/step

Epoch 77/100
639/639 [==============================] - 119s 186ms/step - loss: 0.6538 - val_loss: 1.8726
Epoch 78/100
639/639 [==============================] - 119s 186ms/step - loss: 0.6443 - val_loss: 1.8855
Epoch 79/100
639/639 [==============================] - 119s 187ms/step - loss: 0.6356 - val_loss: 1.9015
Epoch 80/100
639/639 [==============================] - 121s 189ms/step - loss: 0.6423 - val_loss: 1.8863
Epoch 81/100
639/639 [==============================] - 118s 185ms/step - loss: 0.6268 - val_loss: 1.8912
Epoch 82/100
639/639 [==============================] - 126s 197ms/step - loss: 0.6184 - val_loss: 1.9189
Epoch 83/100
639/639 [==============================] - 119s 187ms/step - loss: 0.6075 - val_loss: 1.8962
Epoch 84/100
639/639 [==============================] - 116s 181ms/step - loss: 0.6049 - val_loss: 1.9220
Epoch 85/100
639/639 [==============================] - 117s 183ms/step - loss: 0.6006 - val_loss: 1.9247
Epoch 86/100
639/639 [========================

C:\Users\user\Anaconda3\envs\myvenv\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_5 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_4/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_4/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
C:\Users\user\Anaconda3\envs\myvenv\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_6 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_5/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_5/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
C:\Users\user\Anaconda3\envs\myvenv\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_7 was

-
Input sentence: Who?
Decoded sentence:  이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 
-
Input sentence: Hello!
Decoded sentence: 가가 두이이이이이이    은 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이
-
Input sentence: No way!
Decoded sentence: 가가 두이이이이이이    은 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이
-
Input sentence: No way!
Decoded sentence: 가가 두이이이이이이    은 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이
-
Input sentence: I'm sad.
Decoded sentence:  이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 
-
Input sentence: Me, too.
Decoded sentence:  이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 
-
Input sentence: Perfect!
Decoded sentence: 가가 두이이이이이이    은 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이
-
Input sentence: Shut up!
Decoded sentence:  이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 
-
Input sentence: Welcome.
Decoded sentence:  이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 
-
Input sentence: Welcome.
Decoded sentence:  이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이

-
Input sentence: He was hard up.
Decoded sentence:  이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 
-
Input sentence: I don't buy it.
Decoded sentence:  이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 
-
Input sentence: I like reading.
Decoded sentence:  이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 
-
Input sentence: I love lasagna.
Decoded sentence:  이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 
-
Input sentence: I love my home.
Decoded sentence:  이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 
-
Input sentence: I study Korean.
Decoded sentence:  이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 
-
Input sentence: I'm very sorry.
Decoded sentence:  이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 
-
Input sentence: I'm very sorry.
Decoded sentence:  이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 
-
Input sentence: Keep Tom there.
Decoded sentence:  이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 
-
Input sentence: Only God k